# Calibrate the DM using a Shack-Hartmann wavefront sensor

### Step 1: Installing hcipy on colab.

In [ ]:
!pip uninstall hcipy
!rm -rf hcipy
!git clone https://github.com/ehpor/hcipy.git
!cd hcipy; git pull
!cd hcipy; python setup.py install

### Step 2: Importing necessary libraries.

In [ ]:
# Necessary imports
import gym
from gym import spaces
from gym.utils import seeding
import numpy as np
from hcipy import *
import matplotlib.pyplot as plt
from astropy.io import fits
import os, glob
import time
import tensorflow as tf

### Step 4: Define testbench parameters.

In [ ]:
# Create aperture and pupil/focal grids
wavelength = 532e-9
N = 256
D = 10.5e-3
pupil_grid = make_pupil_grid(N, D*1.1)
science_focal_grid = make_focal_grid(8, 20, wavelength/D)
aperture = circular_aperture(D)

# Create the deformable mirror
num_actuators = 25
xinetics_basis = make_xinetics_influence_functions(pupil_grid, num_actuators, D * 1.1 / num_actuators)
dm = DeformableMirror(xinetics_basis)
num_modes = len(dm.influence_functions)
dm.actuators = np.zeros(num_modes)

## Create propagator from pupil to focal plane
prop = FraunhoferPropagator(pupil_grid, science_focal_grid)

wf = Wavefront(aperture(pupil_grid), wavelength)
wf.total_power = 100000
## Get the unit lambda/D
l_D = wavelength / D
plot_grid = make_focal_grid(8, 20, 1)

## Create the Shack-Hartmann wavefront sensor and estimator
## Create the microlens array
F_mla = 30. / 0.3
N_mla = 32
D_mla = 10.5e-3

shwfs = SquareShackHartmannWavefrontSensorOptics(pupil_grid, F_mla, N_mla, D_mla)
shwfse = ShackHartmannWavefrontSensorEstimator(shwfs.mla_grid, shwfs.micro_lens_array.mla_index, circular_aperture(D)(shwfs.mla_grid).astype('bool'))

## Step 5: Create a function to calibrate the WFS

In [ ]:
def shack_hartmann_calibrator(wf, shwfs, shwfse, dm, amp, prop, filt=None):
    
    # Zero out the DM actuators
    dm.actuators = np.zeros(dm.actuators.shape)
    
    # Get the refernce lenslet measurements
    img = shwfs(wf).power
    ref = shwfse.estimate([img]).ravel()
    num_measurements = ref.shape[0]
    
    Infmat = []
    
    images = []
    
    for dm_mode in np.arange(num_modes):
        
        print("\rNow calibrating actuator {}/{}".format(dm_mode+1, num_modes), end="")
        
        total_slopes = np.zeros((num_measurements,))
        
        for push in np.array([-amp, amp]):
        
            act_levels = np.zeros(num_modes)
            act_levels[dm_mode] = push
            
            dm.actuators = act_levels.copy()
            dm_wf = dm.forward(wf)
            if filt is None:
                sh_wf = shwfs.forward(dm_wf)
            else:
                filt_wf = prop.backward(filt.forward(prop(dm_wf)))
                sh_wf = shwfs.forward(filt_wf)
            sh_img = sh_wf.power
            
            images.append(sh_img.shaped)
            
            lenslet_centers = shwfse.estimate([sh_img])
            total_slopes += (lenslet_centers.ravel()- ref)/(2*push)
        Infmat.append(total_slopes)
        
    dm.actuators = np.zeros(num_modes)
        
    Infmat = ModeBasis(Infmat)
    
    write_fits(np.array(images), "sh_imgs_shaped.fits")
    
    return Infmat

In [ ]:
## Now calibrating!
wfs_infmat = shack_hartmann_calibrator(wf, shwfs, shwfse, dm, 0.01e-6, prop, None)

In [ ]:
control_mat = inverse_tikhonov(wfs_infmat.transformation_matrix, rcond=2e-2)

In [ ]:
np.save("control_mat", control_mat)

## Step 6: Create atmospheric turbulence to test on

In [ ]:
# Telescope parameters
Dtel = 4
tel_pupil_grid = make_pupil_grid(N, Dtel)
tel_aperture = circular_aperture(Dtel)

# Atmosphere parameters
velocity = 10 #m/s
L0 = 40 # outer scale
r0 = 0.2 # Fried parameter
height = 0 # layer height
timestep = 1e-3 # 1 ms per phasescreen

# Make atmosphere
np.random.seed(19900305)
layers = []
layer = InfiniteAtmosphericLayer(tel_pupil_grid, Cn_squared_from_fried_parameter(r0, 500e-9), L0, velocity, height, stencil_length=2, use_interpolation=True)
layers.append(layer)
atmosphere = MultiLayerAtmosphere(layers, False)

## Create a demagnifier
demag = Magnifier(D / Dtel)

# Make initial phasescreen
wf_tel = Wavefront(tel_aperture(tel_pupil_grid), wavelength)
wf_tel.total_power = 100000
wf = demag.forward(wf_tel)

# Generate a diffraction limited image for metrics
diff_lim_img = prop(wf).power

In [ ]:
# Create a new phasescreen
atmosphere.evolve_until(None)
# Make sure lag is accounted for
for loop in np.arange(0.001, 0.1, 0.001):
    atmosphere.evolve_until(loop)

In [ ]:
wfatms_tel = atmosphere.forward(wf_tel)
wfatms = demag.forward(wfatms_tel)

imshow_field(wfatms.phase, vmin=-np.pi, vmax=np.pi, cmap='RdBu')

In [ ]:
# Lets get the proper lenslet measurements we want
ref_img = shwfs.forward(wf).power
ref_slopes = shwfse.estimate([ref_img]).ravel()

In [ ]:
sh_img = shwfs.forward(wfatms).power
meas_vec = (shwfse.estimate([sh_img])).ravel()

In [ ]:
imshow_field(sh_img)

In [ ]:
amplitudes = control_mat.dot(meas_vec-ref_slopes)

In [ ]:
amplitudes -= np.mean(amplitudes)

In [ ]:
np.min(amplitudes) / 1e-6

In [ ]:
np.max(amplitudes) / 1e-6

In [ ]:
amplitudes = np.clip(amplitudes, -1e-6, 1e-6)

In [ ]:
dm.actuators = -amplitudes.copy()

In [ ]:
wfcor = dm.forward(wfatms)
img = prop(wfcor).power

In [ ]:
imshow_field(np.log10(img))

In [ ]:
strehl = img[np.argmax(diff_lim_img)] / diff_lim_img.max()
#strehl = img.max() / diff_lim_img.max()
print(strehl)

In [ ]:
imshow_field(aperture(pupil_grid)*wfcor.phase, vmin=-np.pi, vmax=np.pi, cmap='RdBu')

In [ ]:
for t in range(25):
    dm_wf = dm.forward(wfatms)
    sh_img = shwfs.forward(dm_wf).power
    meas_vec = (shwfse.estimate([sh_img])).ravel()
    amplitudes = control_mat.dot(meas_vec-ref_slopes)
    amplitudes = np.clip(amplitudes, -1e-6, 1e-6)
    dm.actuators -= amplitudes
    wfcor = dm.forward(wfatms)
    img = prop(wfcor).power
    
    strehl = img[np.argmax(diff_lim_img)] / diff_lim_img.max()
    print(strehl)
    
    imshow_field(np.log10(img))
    plt.show()
    imshow_field(wfcor.phase, vmin=-np.pi, vmax=np.pi, cmap='RdBu')
    plt.show()